In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.arima.model import ARIMA
from pycaret.anomaly import *

In [54]:
# Read the data from the CSV file and create a dataframe
df1 = pd.read_csv('datasets/Building_Base_Cooling.csv')
df1

,TIMESTAMP,T_Stair_101,T_Room_102,T_Room_103,T_Room_104,T_Room_105,T_Room_106,T_Stair_201,T_Room_202,T_Room_203,...,AF_VAV_102,AF_VAV_103,AF_VAV_104,AF_VAV_105,AF_VAV_106,AF_VAV_202,AF_VAV_203,AF_VAV_204,AF_VAV_205,AF_VAV_206
0,Unit,Deg C,Deg C,Deg C,Deg C,Deg C,Deg C,Deg C,Deg C,Deg C,...,m3/s,m3/s,m3/s,m3/s,m3/s,m3/s,m3/s,m3/s,m3/s,m3/s
1,7/8/2021 0:00,24.12777778,20.85555556,20.95555556,20.90555556,20.92777778,20.92222222,25.16111111,20.99444444,20.93333333,...,0.054012212,0.066047267,0.14927482,0.164370045,0.144674413,0.084564938,0.071332605,0.163082198,0.134662473,0.15077318
2,7/8/2021 0:01,24.12777778,20.88888889,20.91666667,20.94444444,20.95,20.95,25.15555556,20.98333333,21.03888889,...,0.054182164,0.065846765,0.148246977,0.16313794,0.144730108,0.084229076,0.071792312,0.163800452,0.13658811,0.148476102
3,7/8/2021 0:02,24.11666667,20.88333333,20.89444444,20.98333333,20.96111111,20.94444444,25.14444444,20.96666667,21.05555556,...,0.054001966,0.065559783,0.14893454,0.163590221,0.144513673,0.084294218,0.071913196,0.163695501,0.135801988,0.149739462
4,7/8/2021 0:03,24.11111111,20.92777778,20.89444444,20.99444444,20.99444444,20.95555556,25.13888889,20.96666667,21.01666667,...,0.054306573,0.065857763,0.147553774,0.165590118,0.144526081,0.084765581,0.073009142,0.166611757,0.136569733,0.151312129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10076,7/14/2021 23:55,23.833333,21.061111,21.05,21.027778,21.072222,21.038889,25.227778,20.916667,20.872222,...,0.054936,0.067265,0.15103,0.168205,0.144975,0.084162,0.071108,0.165927,0.136399,0.152856
10077,7/14/2021 23:56,23.827778,21.0,20.988889,20.988889,21.005556,21.005556,25.222222,20.961111,20.827778,...,0.054989,0.067335,0.150553,0.167388,0.145272,0.084227,0.071456,0.165078,0.135186,0.152466
10078,7/14/2021 23:57,23.822222,20.933333,20.983333,20.988889,20.988889,20.955556,25.216667,20.972222,20.894444,...,0.054825,0.066847,0.151327,0.16844,0.145668,0.084297,0.071057,0.164001,0.136653,0.15093
10079,7/14/2021 23:58,23.827778,20.905556,20.944444,20.938889,20.927778,20.905556,25.211111,20.966667,21.005556,...,0.054412,0.066717,0.150256,0.165815,0.145439,0.084227,0.07134,0.163865,0.13482,0.151246


In [55]:
# Read the data from the CSV file and create a dataframe
df2 = pd.read_csv('datasets/Weather_Base_Cooling.csv')
df2

,TIMESTAMP,T_out,RH_out,BP,Dir_Solar,Dif_Solar,Glo_Solar,WS,WD
0,Unit,Deg C,%,Pa,W/m2,W/m2,W/m2,m/s,Deg
1,7/8/2021 0:00,20.23333333,99.6,98377.8156,0.37,-0.046,-1.358,0,0
2,7/8/2021 0:01,20.23333333,99.6,98309.068,0.423,-0.046,-1.308,0,0
3,7/8/2021 0:02,20.25555556,99.6,98309.068,0.37,0.046,-1.257,0,0
4,7/8/2021 0:03,20.21666667,99.6,98309.068,0.423,0,-1.308,0,0
...,...,...,...,...,...,...,...,...,...
10076,7/14/2021 23:55,21.53888889,100,98996.544,0.212,0.046,-1.962,0,0
10077,7/14/2021 23:56,21.42222222,100,98996.544,0.212,0,-2.163,0,0
10078,7/14/2021 23:57,21.48333333,100,98996.544,0.212,0.046,-2.263,0,0
10079,7/14/2021 23:58,21.55555556,100,98996.544,0.212,-0.046,-2.314,0,0


In [56]:
new_df = df1.loc[:, ['T_Room_106','T_VAV_106']]
new_df

,T_Room_106,T_VAV_106
0,Deg C,Deg C
1,20.92222222,18.52222222
2,20.95,18.17777778
3,20.94444444,18.59444444
4,20.95555556,18.30555556
...,...,...
10076,21.038889,16.277778
10077,21.005556,15.966667
10078,20.955556,16.955556
10079,20.905556,18.372222


In [57]:
df = pd.concat([df2, new_df], axis=1, join='outer')
df

,TIMESTAMP,T_out,RH_out,BP,Dir_Solar,Dif_Solar,Glo_Solar,WS,WD,T_Room_106,T_VAV_106
0,Unit,Deg C,%,Pa,W/m2,W/m2,W/m2,m/s,Deg,Deg C,Deg C
1,7/8/2021 0:00,20.23333333,99.6,98377.8156,0.37,-0.046,-1.358,0,0,20.92222222,18.52222222
2,7/8/2021 0:01,20.23333333,99.6,98309.068,0.423,-0.046,-1.308,0,0,20.95,18.17777778
3,7/8/2021 0:02,20.25555556,99.6,98309.068,0.37,0.046,-1.257,0,0,20.94444444,18.59444444
4,7/8/2021 0:03,20.21666667,99.6,98309.068,0.423,0,-1.308,0,0,20.95555556,18.30555556
...,...,...,...,...,...,...,...,...,...,...,...
10076,7/14/2021 23:55,21.53888889,100,98996.544,0.212,0.046,-1.962,0,0,21.038889,16.277778
10077,7/14/2021 23:56,21.42222222,100,98996.544,0.212,0,-2.163,0,0,21.005556,15.966667
10078,7/14/2021 23:57,21.48333333,100,98996.544,0.212,0.046,-2.263,0,0,20.955556,16.955556
10079,7/14/2021 23:58,21.55555556,100,98996.544,0.212,-0.046,-2.314,0,0,20.905556,18.372222


In [58]:
# Delete the first row
second_row = df.iloc[0]

new_columns = []
for i in range(len(df.columns)):
    new_columns.append(df.columns[i] + '_' + str(second_row[i]))
df.columns = new_columns
df = df.drop(df.index[0])
df

,TIMESTAMP_Unit,T_out_Deg C,RH_out_%,BP_Pa,Dir_Solar_W/m2,Dif_Solar_W/m2,Glo_Solar_W/m2,WS_m/s,WD_Deg,T_Room_106_Deg C,T_VAV_106_Deg C
1,7/8/2021 0:00,20.23333333,99.6,98377.8156,0.37,-0.046,-1.358,0,0,20.92222222,18.52222222
2,7/8/2021 0:01,20.23333333,99.6,98309.068,0.423,-0.046,-1.308,0,0,20.95,18.17777778
3,7/8/2021 0:02,20.25555556,99.6,98309.068,0.37,0.046,-1.257,0,0,20.94444444,18.59444444
4,7/8/2021 0:03,20.21666667,99.6,98309.068,0.423,0,-1.308,0,0,20.95555556,18.30555556
5,7/8/2021 0:04,20.23333333,99.5,98309.068,0.423,0.092,-1.308,0,0,20.96666667,18.36666667
...,...,...,...,...,...,...,...,...,...,...,...
10076,7/14/2021 23:55,21.53888889,100,98996.544,0.212,0.046,-1.962,0,0,21.038889,16.277778
10077,7/14/2021 23:56,21.42222222,100,98996.544,0.212,0,-2.163,0,0,21.005556,15.966667
10078,7/14/2021 23:57,21.48333333,100,98996.544,0.212,0.046,-2.263,0,0,20.955556,16.955556
10079,7/14/2021 23:58,21.55555556,100,98996.544,0.212,-0.046,-2.314,0,0,20.905556,18.372222


In [59]:
# Change data types to datetime and float
df.columns = new_columns
df = df.drop(df.index[0])
df = df.rename(columns={df.columns[0]: 'timestamp'})
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%m/%d/%Y %H:%M')
df.iloc[:, 1:] = df.iloc[:, 1:].astype(float)
df = df.set_index('timestamp')
df

,T_out_Deg C,RH_out_%,BP_Pa,Dir_Solar_W/m2,Dif_Solar_W/m2,Glo_Solar_W/m2,WS_m/s,WD_Deg,T_Room_106_Deg C,T_VAV_106_Deg C
timestamp,,,,,,,,,,
2021-07-08 00:01:00,20.233333,99.6,98309.068,0.423,-0.046,-1.308,0.0,0.0,20.950000,18.177778
2021-07-08 00:02:00,20.255556,99.6,98309.068,0.370,0.046,-1.257,0.0,0.0,20.944444,18.594444
2021-07-08 00:03:00,20.216667,99.6,98309.068,0.423,0.000,-1.308,0.0,0.0,20.955556,18.305556
2021-07-08 00:04:00,20.233333,99.5,98309.068,0.423,0.092,-1.308,0.0,0.0,20.966667,18.366667
2021-07-08 00:05:00,20.233333,99.6,98309.068,0.423,0.046,-1.358,0.0,0.0,20.983333,19.094444
...,...,...,...,...,...,...,...,...,...,...
2021-07-14 23:55:00,21.538889,100.0,98996.544,0.212,0.046,-1.962,0.0,0.0,21.038889,16.277778
2021-07-14 23:56:00,21.422222,100.0,98996.544,0.212,0.000,-2.163,0.0,0.0,21.005556,15.966667
2021-07-14 23:57:00,21.483333,100.0,98996.544,0.212,0.046,-2.263,0.0,0.0,20.955556,16.955556


In [60]:
df_train = df.iloc[:-1000]
df_test = df.tail(1000)

anom = setup(data = df_train,
             silent = True)

anom_model = create_model(model = 'iforest', fraction = 0.05)
results = assign_model(anom_model)


In [61]:
plot_model(anom_model, plot = 'tsne')


In [62]:
plot_model(anom_model, plot = 'umap')


In [63]:
anom_model.predict(df_test)


array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [64]:
anom_model.predict_proba(df_test)

array([[0.31300321, 0.68699679],
       [0.28667629, 0.71332371],
       [0.36403405, 0.63596595],
       ...,
       [0.73195527, 0.26804473],
       [0.70327272, 0.29672728],
       [0.716789  , 0.283211  ]])

In [65]:
anom_model.decision_function(df_test)


array([ 1.00253121e-02,  1.72684432e-02, -4.01443034e-03,  6.37128635e-03,
        6.53764624e-03,  2.71149889e-02,  2.96537344e-02,  1.62238083e-02,
        1.44016253e-02,  1.27762995e-02,  1.48483907e-02,  3.15616655e-02,
        3.42632560e-02,  3.47986407e-02,  3.00292544e-02,  3.14149293e-02,
        3.21858039e-02,  3.29623545e-02,  3.25443885e-02,  2.91857953e-02,
        1.48184504e-02,  4.43837731e-03,  4.97903719e-03,  9.44075906e-03,
       -4.95980899e-04,  1.89513645e-02,  1.15599979e-02,  2.62968464e-02,
        9.85600287e-03,  2.33581939e-02,  2.61287323e-02,  2.35003339e-02,
        2.80484088e-03,  5.99982150e-03, -8.47034136e-04, -2.45852908e-02,
       -1.07162056e-02, -2.81200089e-02, -3.21812877e-02, -2.14003213e-02,
       -1.92367439e-02, -6.49155521e-04,  1.78352098e-02, -1.60785538e-02,
       -2.46866302e-02, -9.33276256e-03, -1.33310006e-02, -6.54407676e-03,
        8.67410324e-04,  5.02839327e-03,  1.62156765e-02,  1.92353094e-02,
        4.89766636e-03,  